In [1]:
import fitz  # PyMuPDF
from transformers import pipeline

/home/arunav/.conda/envs/tf-gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-13 22:11:05.820475: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-13 22:11:06.069835: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-13 22:11:07.140891: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warni

In [18]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text()
    return text

def chunk_text(text, max_chunk_size=1000):
    chunks = []
    current_chunk = ""
    for word in text.split():
        if len(current_chunk) + len(word) + 1 > max_chunk_size:
            chunks.append(current_chunk)
            current_chunk = word
        else:
            if current_chunk:
                current_chunk += " " + word
            else:
                current_chunk = word
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

def summarize_text(chunks, summarizer):
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=50, min_length=10, do_sample=False)
        summaries.append(summary[0]['summary_text'])
    return " ".join(summaries)

def generate_title(text):
    summarizer = pipeline("summarization", model="google/pegasus-xsum", device=0)
    chunks = chunk_text(text)
    summary = summarize_text(chunks, summarizer)
    title = summarizer(summary, max_length=15, min_length=5, do_sample=False)
    return title[0]['summary_text']

In [19]:

def main(pdf_path):
    text = extract_text_from_pdf(pdf_path)
    title = generate_title(text)
    return title

if __name__ == "__main__":
    pdf_path = "/home/arunav/Downloads/jeff103.pdf"
    title = main(pdf_path)
    print("Generated Title:", title)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generated Title: In our series of letters from African journalists, novelist and writer Col
